In [2]:
!pip install QuantumRingsLib
!pip install quantumrings-toolkit-qiskit
!pip install qiskit==1.3.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.7/51.7 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 81.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.7/119.7 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 54.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 MB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.0/109.0 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 71.3 MB/s eta 0:00:00
  Attempting uninstall: qiskit
    Found existing installation: qiskit 2.0.0
    Uninstalling qiskit-2.0.0:
      Successfully uninstalled qiskit-2.0.0


In [3]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit.circuit.library import DraperQFTAdder
import QuantumRingsLib
from QuantumRingsLib import QuantumRingsProvider
from quantumrings.toolkit.qiskit import QrBackendV2, QrJobV1

In [158]:
class QuantumEnv():
    def __init__(self, qubits, cbits):
        self.qsize = qubits
        self.csize = cbits

        self.qfree = list(range(qubits))
        self.cfree = list(range(cbits))

        self.qc = QuantumCircuit(qubits, cbits)
        self.vars = {}
    def qalloc(self, n):
        if (len(self.qfree) < n):
            raise MemoryError(f"Requested {n} qubits, {len(self.qfree)} available. Try freeing unused qubits (using del) or initializing the QuantumEnv with more qubits.")
        else:
            allocated = self.qfree[:n]
            self.qfree = self.qfree[n:]
            return allocated

    def calloc(self, n):
        if (len(self.cfree) < n):
            raise MemoryError(f"Requested {n} classical bits, {len(self.cfree)} available. Preallocate more classical bits during QuantumEnv initialization.")
        else:
            allocated = self.cfree[:n]
            self.cfree = self.cfree[n:]
            return allocated

    def qdealloc(self, qubits):
        self.qfree += qubits
        self.qc.reset(qubits)

    def process_output(self, bitstring, returnBinary = False):
        results = {}
        for var, vardata in self.vars.items():
          if not vardata['measured']:
            results[var] = 'Unmeasured'
          else:
            if returnBinary:
              results[var] = QubitArray.format_bitstring("".join(bitstring[bit] for bit in vardata['cbit_locs'][::-1]))
            else:
              results[var] = vardata['type'].format_bitstring("".join(bitstring[bit] for bit in vardata['cbit_locs'][::-1]))
        return results

class QubitArray():
    def __init__(self, env, size, name, val=0):
        self.env = env
        self.size = size
        self.name = name
        self.qubits = self.env.qalloc(size)
        self.cbits = self.env.calloc(size)
        self.env.vars[name] = {
            "measured": False,
            "cbit_locs": self.cbits,
            "type": self.__class__
        }
        self.set(val)

    def clone(self, orginal, clone_name):
        result = QubitArray(self.env, self.size, clone_name, val=0)
        for o, n in zip(self.qubits, result.qubits):
            self.env.cx(o, n)
        return result

    def superposition(self):
        for qubit in self.qubits:
          self.env.qc.h(qubit)

    def measure(self):
        self.env.qc.measure(self.qubits, self.cbits)
        self.env.vars[self.name]['measured'] = True

    def reset(self):
        self.env.qc.reset(self.qubits)

    def set(self, val):
        self.reset()
        if len(bin(val)[2:]) > self.size:
            raise ValueError(f"Error: QubitArray cannot be set to {val} as it requires {len(bin(val)[2:])} bits while the QubitArray only contains {self.size} bits")
        for ind, bit in enumerate(bin(val)[2:].zfill(self.size)[::-1]):
            if int(bit) == 1:
                self.env.qc.x(self.qubits[ind])

    def __del__(self):
        self.env.qdealloc(self.qubits)
        del self.env.vars[self.name]

    @staticmethod
    def format_bitstring(bitstring):
      return bitstring

class QBool(QubitArray):
    def __init__(self, env, name, val=0):
        super().__init__(env, 1, name, int(val))
    def inverse(self):
        self.env.qc.x(self.qubits)
    @staticmethod
    def format_bitstring(bitstring):
        return bitstring == "1"

class QInt(QubitArray):
    def __init__(self, env, size, name, val=0):
        super().__init__(env, size, name, val)
    def __iadd__(self, other):
        if not isinstance(other, QInt):
            raise TypeError(f"Error: Can't add QInt with {type(other)}")
        elif other.env != self.env:
            raise TypeError("Error: Can only add between QInts from the same QuantumEnv")
        elif other.size != self.size:
            raise ValueError("Error: Can't add between two QInts with different sizes")
        else:
            self.env.qc.append(DraperQFTAdder(self.size, kind='fixed', name='DraperQFTAdder').to_gate(),[*other.qubits, *self.qubits])
        return self
    def negate(self):
      self.env.qc.x(self.qubits)
      one = QInt(self.env, self.size, 'one', 1)
      self += one
      del one

    def __isub__(self, other):
      other.negate()
      self += other
      other.negate()
      return self

    @staticmethod
    def format_bitstring(bitstring):
      return int(bitstring, 2)

def countsToBitstring(counts):
  return list(counts.keys())[0][::-1]

In [161]:
#Example Code
qenv = QuantumEnv(12, 16) #initialize a Quantum Environment with 8 qubits and 8 classical bits. The extra space is for the 8 classical qubits required during getting complement
a = QInt(qenv, size=4, name='a', val=3) #defining two integers, a and b, both 4 qubits large.
b = QInt(qenv, size=4, name='b', val=5) #a is set to 3, b is set to 5
b -= a #substracting a from b in-place (b = b - a = 5 - 3 = 2)
a += b #adding b to a, in-place again (a = a + b = 3 + 2 = 5)
a.measure() #reading both values
b.measure()

#this logic is to use QuantumRings to execute the circuit
#will try to find a way to make this be cleaner for users
qr_provider = QuantumRingsProvider(token ="rings-200.inm2AQBhP6HljeZBxhFso18iyRr9F8ZX", name="manyatapathania@gmail.com")
mybackend = QrBackendV2(qr_provider, num_qubits = qenv.qc.num_qubits)
qc_transpiled = transpile(qenv.qc, mybackend, initial_layout=[i for i in range(0, qenv.qc.num_qubits)])
job = mybackend.run(qc_transpiled, shots = 1)
result = job.result()
counts = result.get_counts()
#from here we go back to functions from QLang

bitstring = countsToBitstring(counts) #getting a bitstring of the final classical register output
print(qenv.process_output(bitstring)) #converting the bitstring into mappings of varname : value
                                      #process_output also converts binary to appropriate types (if it's a QInt, it would be in base10, QBool would be represented by a boolean, etc)

{'a': 5, 'b': 2}
